### Exercises

#### CRUD operations:


**TODO 1**: You will continue to work on the MCU network for a while. Who are the 10 fastest in the MCU ? Are they related to each other ?


In [38]:
import pandas as pd
from py2neo import Graph, Node, Relationship

characters_csv = "/Users/valen/Desktop/data/Chap4/Marvel_Superheroes/characters.csv"
characters_stats_csv = "/Users/valen/Desktop/data/Chap4/Marvel_Superheroes/characters_stats.csv"
characters2comics_csv = "/Users/valen/Desktop/data/Chap4/Marvel_Superheroes/charactersToComics.csv"

characters = pd.read_csv(characters_csv)
characters_stats = pd.read_csv(characters_stats_csv)
characters2comics = pd.read_csv(characters2comics_csv)


In [59]:
characters_stats

,Name,Alignment,Intelligence,Strength,Speed,Durability,Power,Combat,Total
0,3-D Man,good,50,31,43,32,25,52,233
1,A-Bomb,good,38,100,17,80,17,64,316
2,Abe Sapien,good,88,14,35,42,35,85,299
3,Abin Sur,good,50,90,53,64,84,65,406
4,Abomination,bad,63,80,53,90,55,95,436
...,...,...,...,...,...,...,...,...,...
606,Yellowjacket,good,88,10,12,28,12,14,164
607,Yellowjacket II,good,50,10,35,28,31,28,182
608,Ymir,good,50,100,27,100,83,28,388
609,Zatanna,good,75,10,23,28,100,56,292


In [32]:
import pandas as pd
import tqdm
import py2neo

graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"))

graph.delete_all()

try:
    graph.run("CREATE CONSTRAINT ON (n:Superhero) ASSERT n.id_ IS UNIQUE")
except:
    print("Constraint already exists")


transaction_list = []

for row in tqdm.tqdm(characters.iterrows()):
    id_ = row[1]["characterID"]
    name = row[1]["name"]
    transaction_list.append({"id_":id_,"name":name})
    if len(transaction_list) == 1000:
        transaction = "UNWIND $json as data CREATE (n:Superhero) SET n = data"
        graph.run(transaction, json=transaction_list)
        transaction_list = []

graph.run(transaction, json=transaction_list)

pass

Constraint already exists


1170it [00:00, 6594.16it/s]


In [35]:
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"))

transaction = """UNWIND $json as data
OPTIONAL MATCH (a:Superhero {name:data.Name})
SET a += {Alignement:data.Alignement,Intelligence:data.Intelligence,Strength:data.Strength,Speed:data.Speed,Durability:data.Durability,
         Power:data.Power,Combat:data.Combat,Total:data.Total} 
"""

transaction_list = []

iteration = 0
for row in tqdm.tqdm(characters_stats.iterrows()):
    transaction_list.append(dict(row[1]))
    iteration += 1
    if iteration % 1000 == 0:
        graph.run(transaction, json=transaction_list)
        transaction_list = []
        
graph.run(transaction, json=transaction_list)

pass

0it [00:00, ?it/s]

611it [00:00, 17057.85it/s]


In [41]:
import pandas as pd
import tqdm
import py2neo
import itertools

graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"))

characters2comics = characters2comics.drop_duplicates()
transaction_list = []

transaction = """UNWIND $json as data
MATCH (a:Superhero),(b:Superhero)
WHERE a.id_ = data.id_1 AND b.id_ = data.id_2
MERGE (a)-[r:RELATION]->(b)
ON CREATE
    SET r.n_collab = 1
ON MATCH
    SET r.n_collab = r.n_collab + 1
"""

it = 0   
for comic in tqdm.tqdm(characters2comics.groupby(["comicID"])):
    related_ids = comic[1]["characterID"]
    for combi in list(itertools.combinations(related_ids, 2)):
        sh1 = combi[0]
        sh2 = combi[1]
        transaction_list.append({"id_1":sh1,"id_2":sh2})
        it +=1
        if it % 1000 == 0:
            graph.run(transaction, json=transaction_list)
            transaction_list = []

graph.run(transaction, json=transaction_list)
pass

100%|██████████| 22250/22250 [05:04<00:00, 73.01it/s] 


In [42]:
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"))

query = """MATCH (a:Superhero{name:'Pepper Potts'})-[r:RELATION]-(b:Superhero{name:'Iron Man'}) RETURN r,a,b"""
data = graph.run(query).to_data_frame()
print(data["r"],data["a"],data["b"])

0    {'n_collab': 68}
Name: r, dtype: object 0    {'id_': 1009494, 'name': 'Pepper Potts'}
Name: a, dtype: object 0    {'Speed': 58, 'Intelligence': 100, 'id_': 1009...
Name: b, dtype: object


In [53]:
#Who are the 10 fastest in the MCU ?
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"))


query = """
MATCH (c:Superhero)
WHERE c.Speed IS NOT NULL
RETURN c.name AS name, c.Speed AS speed
ORDER BY c.Speed DESC
LIMIT 10
"""

fastest_names = graph.run(query).to_data_frame()
fastest_names


,name,speed
0,Black Bolt,100
1,Stardust,100
2,Quicksilver,100
3,Nova,100
4,Thor,92
5,Silver Surfer,84
6,Galactus,83
7,Dormammu,83
8,Ego,83
9,Northstar,83


In [55]:

fastest_characters = graph.run(query).data()
fastest_names = [char['name'] for char in fastest_characters]

query_relationships = """
MATCH (c1:Superhero)-[r]-(c2:Superhero)
WHERE c1.name IN $names AND c2.name IN $names
RETURN c1.name AS source, c2.name AS target, type(r) AS relationship
"""

relationships = graph.run(query_relationships, names=fastest_names).data()

for rel in relationships:
    print(f"{rel['source']} is related to {rel['target']} via {rel['relationship']}")


Galactus is related to Ego via RELATION
Galactus is related to Dormammu via RELATION
Galactus is related to Black Bolt via RELATION
Galactus is related to Northstar via RELATION
Galactus is related to Nova via RELATION
Galactus is related to Stardust via RELATION
Galactus is related to Silver Surfer via RELATION
Galactus is related to Quicksilver via RELATION
Galactus is related to Thor via RELATION
Silver Surfer is related to Ego via RELATION
Silver Surfer is related to Nova via RELATION
Silver Surfer is related to Quicksilver via RELATION
Silver Surfer is related to Black Bolt via RELATION
Silver Surfer is related to Galactus via RELATION
Silver Surfer is related to Dormammu via RELATION
Silver Surfer is related to Stardust via RELATION
Silver Surfer is related to Thor via RELATION
Thor is related to Northstar via RELATION
Thor is related to Ego via RELATION
Thor is related to Silver Surfer via RELATION
Thor is related to Nova via RELATION
Thor is related to Black Bolt via RELATION
T

In [57]:
#Produisons une matrice pour observons les relations
matrix = pd.DataFrame(0, index=fastest_names, columns=fastest_names)

for rel in relationships:
    matrix.at[rel['source'], rel['target']] = 1
    matrix.at[rel['target'], rel['source']] = 1  

matrix

,Black Bolt,Stardust,Quicksilver,Nova,Thor,Silver Surfer,Galactus,Dormammu,Ego,Northstar
Black Bolt,0,0,1,1,1,1,1,0,0,0
Stardust,0,0,0,0,0,1,1,0,0,0
Quicksilver,1,0,0,1,1,1,1,1,1,1
Nova,1,0,1,0,1,1,1,0,1,0
Thor,1,0,1,1,0,1,1,1,1,1
Silver Surfer,1,1,1,1,1,0,1,1,1,0
Galactus,1,1,1,1,1,1,0,1,1,1
Dormammu,0,0,1,0,1,1,1,0,0,0
Ego,0,0,1,1,1,1,1,0,0,0
Northstar,0,0,1,0,1,0,1,0,0,0


**TODO 2**: How many have collaborated atleast more than 10 times with another superhero?



In [58]:
query = """
MATCH (c1:Superhero)-[r:RELATION]-(c2:Superhero)
RETURN count(DISTINCT c1.id_ + c2.id_) AS collaboration_count
"""

collaboration_count = graph.run(query).evaluate()
collaboration_count

4907

**TODO 3**: Did the weakest of the MCU collaborated with the strongest ?


In [66]:
#Pour déterminer le "strongest" nous nous baserons sur la caractéristique "Strength", nous pourrions cependant tout aussi bien nous
#intéressé à la somme des caractéristiques du perso.
query_strength = """
MATCH (c:Superhero)
WHERE c.Strength IS NOT NULL
RETURN c.name AS name, c.Strength AS strength
ORDER BY c.Strength DESC
LIMIT 1
"""

query_weakest = """
MATCH (c:Superhero)
WHERE c.Strength IS NOT NULL
RETURN c.name AS name, c.Strength AS strength
ORDER BY c.Strength ASC
LIMIT 1
"""

strongest_character = graph.run(query_strength).evaluate('name')
weakest_character = graph.run(query_weakest).evaluate('name')

query_collaboration = """
MATCH (c1:Superhero)-[r:RELATION]-(c2:Superhero)
WHERE c1.name = $strongest AND c2.name = $weakest
RETURN count(r) > 0 AS collaborated
"""

collaborated = graph.run(query_collaboration, strongest=strongest_character, weakest=weakest_character).evaluate()
print(strongest_character)
print(weakest_character)
print(collaborated)


Galactus
Lady Deathstrike
False


        -> Non le plus fort et le plus faible n'ont pas collaboré ensemble

**TODO 4**: Give a stats that interests you !


In [67]:
query_top_pairs = """
MATCH (c1:Superhero)-[r:RELATION]-(c2:Superhero)
RETURN c1.name AS superhero1, c2.name AS superhero2, r.n_collab AS collaborations
ORDER BY collaborations DESC
LIMIT 10
"""

# Execute the query and fetch results
top_pairs = graph.run(query_top_pairs).to_data_frame()

# Print the results
print(top_pairs)


        superhero1       superhero2  collaborations
0        Wolverine            X-Men             805
1            X-Men        Wolverine             805
2        Wolverine         Iron Man             721
3         Iron Man        Wolverine             721
4          Cyclops            X-Men             676
5            X-Men          Cyclops             676
6         Iron Man  Captain America             639
7  Captain America         Iron Man             639
8            Storm            X-Men             572
9            X-Men            Storm             572


**TODO 5**: Create a new db called "master". Create a graph from the M1 class with name and random age between 20 and 30, Group of 3 that collab (undirected relation).


**TODO 6**: Get the age for people YOU collaborate with.


**TODO 7**: Get nodes with age > 25.

### Real world problem:

**TODO 8**: Using the downloaded data in TODO 11 of chapter II, create a collaboration network between authors for papers after 2021 (same author if same name). Insert it in neo4j in a db called "arxiv".

**TODO 9**: Get the density of degrees of the collaboration network

**TODO 10**: Do a for loop for i in range(500) for each i remove nodes with a degree less than i.

### Homework 3

You will work again on [pubmed_last_100k.json](https://we.tl/t-6afLCDGRsg). The goal in this homework is to insert collaboration data in Neo4j and then study it using cypher queries. The following questions are meant to guide you through the process.

1) Each paper has a list of author and affiliations. The first step is to create a node per author with a list of affiliation as attribute. If a name appears multiple times consider it as a single person. Create an edge between two authors if they are on the same paper. You can use any kind of operation to clean the data before inserting, you decide which preprocessing step is the most convenient.

2) How many authors have more than 2 affiliations ?

3) Create a density of node degrees. Do you observe a power scale law ? What does it mean ?

4) Which country has seen the most international collaboration ?

5) Who are the top 10 collaborators and what is their affiliation ?

6) Sandbox exercise: think of a problematic and try to answer it.

### Homework 4

change the database "neo4j" to the database "movie". Go to the browser and run on the browser console :play movie-graph then copy the queries on the console and run it. Congrats you just imported a brand new sample of data. Answer the following questions with cypher queries.

1) How many actors played in more than 2 movie ? Get their names and year they were born.

2) In how many movies did Tom hanks play ?

3) In average in how many movies do actors play in ?

4) In average how many actors are there in a movie ?

5) In average how many writers is needed to write a movie ?

6) What is the proportion of writers that also directed the movie ?

7) Do an histogram of the year of release for the movies.

8) Who are the 10 actors with the most movie played in

9) Delete every movie made by a director of your choosing

10) Give some statistics that you find interesting.

11) Sandbox exercise: think of a problematic and try to answer it.

## Homework 5

You will work on a Twitch dataset found [here](https://www.kaggle.com/andreagarritano/twitch-social-networks). The goal is to learn more about the feat&fun on Twitch. First thing to do is create the network on Neo4j. Insert the node
from Twitch/FR_target and then the edges from Twitch/FR_edges.csv in neo4j. Answer the following using cypher queries

1) Give the IDs of the top 10 streamers that have atleast one collaboration.

2) Create a density of node degrees. Do you observe a power scale law ? What does it mean ?

3) How many streamers has the tag "mature" ?

4) Is the collaboration correlated to the "mature" tag ?

5) Do partners collaborate more than non partners ?

6) Sandbox exercise: think of a problematic and try to answer it.

## Homework 6 

You will work on a stackoverflow dataset found [here](https://www.kaggle.com/stackoverflow/stack-overflow-tag-network). First thing to do is create the network on Neo4j. You can find the characteristics of the nodes in stack_network_nodes.csv.
Once the node are create you can create the edges between them, you can find them in stack_network_links.csv. The weight of the edge is the column value. Answer the following making use of cypher queries.

1) What are the 10 most popular keywords ?

2) What are the top 10 Co-occurence ?

3) How many keywords are in group 1 ?

3) Is the Co-occurence correlated to the "group" of the keyword ?  

4) Sandbox exercise: think of a problematic and try to answer it.